# Setup Web Automation with Selenium
Subway Surfers is available on Poki.com. Using Selenium to launch the game and interact with it.


In [2]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
import cv2
import mediapipe as mp
import pyautogui

# Setup browser driver
driver = webdriver.Chrome()
driver.get("https://poki.com/en/g/temple-run-2")
time.sleep(5)  # Allow game page to load

# Ensure the game window is focused
game_window = driver.find_element("tag name", "body")
game_window.click()

# Capture Hand Gestures Map Gestures and Integrate Everything

Using OpenCV & MediaPipe to detect fingers


In [3]:
# Initialize MediaPipe Hands
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(max_num_hands=1, min_detection_confidence=0.85)  # Higher confidence for accuracy
mp_draw = mp.solutions.drawing_utils

# Increase webcam resolution for better detection
cap = cv2.VideoCapture(0)
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

# Reduce lag by processing fewer frames
frame_skip = 3  # Process every 3rd frame

# Function to map finger count to game actions
def perform_action(finger_count):
    actions_dict = {
        1: 'up',    # Jump
        2: 'down',  # Roll
        3: 'right', # Move Right
        4: 'left'   # Move Left
    }

    if finger_count in actions_dict:
        pyautogui.press(actions_dict[finger_count])  # Send real keypress via PyAutoGUI

# Main loop for gesture detection and game control
frame_count = 0

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame_count += 1
    if frame_count % frame_skip != 0:  # Skip some frames to improve performance
        continue

    frame = cv2.flip(frame, 1)
    img_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(img_rgb)

    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            mp_draw.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)

            # Enhanced finger detection logic
            finger_count = sum([1 for idx, lm in enumerate(hand_landmarks.landmark) 
                                if idx in [8, 12, 16, 20] and lm.y < hand_landmarks.landmark[idx - 2].y])

            # Perform game action smoothly
            perform_action(finger_count)
            time.sleep(0.05)  # Short delay to avoid input spamming

            # Display detected fingers
            cv2.putText(frame, f"Fingers: {finger_count}", (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    cv2.imshow("Gesture-Controlled Temple Run 2", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):  # Quit on 'q'
        break

cap.release()
cv2.destroyAllWindows()